# Analysis

Notebook reads the list of the selected authors from `res/selected/author_list` and loads their text data from book and genereted resources available in `res` directory.
Displayed data:
- Notebook displays the number of words per each source-author pair.
- Analysis on chunk of `configuration.analysis_size` size
    - Average word length
    - Average sentence length
    - Unique words count
    - Top 10 function words

Further work on that notebook is planned.

*Note: the list of function words are taken from [functionwords](https://pypi.org/project/functionwords/) package.*

In [ ]:
from src import *

In [15]:
configuration = Configuration()

In [16]:
authors = []
for author_name in FileUtils.read_authors(configuration.selected_authors_filepath):
    author = Author(name=author_name)
    author.read_book_collection(configuration.all_book_data_directory)
    author.read_generated_texts(configuration.model_dirs)
    authors.append(author)

In [17]:
analysis = Analysis(configuration.analysis_size)
analysis_data = analysis.analyze(authors)

In [20]:
AnalysisVisualization().visualize(analysis_data)